In [1]:
!pip install gym
!pip install folium

In [2]:
schema_path = r"/home/johaanto/work/Constrained-EVlearn/data-test/Tiago_dataset/Tiagoschema.json"
schema_test = r"/home/johaanto/work/Constrained-EVlearn/data-test/Test/schema.json"

In [3]:
import os

schema_path = r"/home/johaanto/work/Constrained-EVlearn/data-test/Test/schema.json"
print("File exists:", os.path.exists(schema_path))  # Should print True


File exists: True


In [4]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.__version__)


True
2.2.2


In [5]:
import numpy as np
import pandas as pd

In [6]:
import sys
sys.path.append("./citylearn")

In [ ]:
from citylearn.citylearn import CityLearnEnv
from citylearn.agents.EVs.maddpgMODIFIED import MADDPGOptimizedRBC as RLAgent
from citylearn.reward_function import V2GPenaltyReward

# Load the environment
reward_function = V2GPenaltyReward
env = CityLearnEnv(schema_path, central_agent=False, reward_function=reward_function)

# Define the agent
model = RLAgent(
    env,
    critic_units=[512, 256, 128],
    actor_units=[256, 128, 64],
    lr_actor=0.0006343946342268605,
    lr_critic=0.0009067117952187151,
    gamma=0.9773507798877807,
    sigma=0.2264587893937525,
    lr_dual = 1e-5,
    steps_between_training_updates=20,
    target_update_interval=100
)

# Train the agent
#episodes = 1
#episodes = 1
#model.learn(episodes=episodes)

# Call the learn method to train for, say, 10 episodes:
rewards_all, average_runtime, kpis_list, observations_ep, constraint_all, q_critic_all, lambda_all = model.learn(
    episodes=50,
    deterministic=False,          # Use stochastic actions during training
    deterministic_finish=False,   # You can also set this true if you want the final episode to be deterministic
    keep_env_history=True         # Save the environment state at the end of each episode
)

# Simulation and Logging
log_data = []
violations_count = {
    "Overcharging Violation": 0,
    "Charging with No EV Connected": 0,
    "Charging Power Violation": 0,
    "Discharge During Idle": 0,
    "Discharging when SOC is 0": 0
}


2025-03-09 23:01:57.449506: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 23:01:57.503316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 23:01:57.567460: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 23:01:57.595748: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 23:01:57.646525: I tensorflow/core/platform/cpu_feature_guar

cuda
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
returned due to buffer
return

In [ ]:
def save_metric(metrics, filename):
    import json
    
    with open(filename, "w") as f:
        f.write(json.dumps(metrics, default=str))  

save_metric(constraint_all, "cons.json")
save_metric(lambda_all, "lambda_all.json")
save_metric(q_critic_all, "q_critic_all")
print("constraint_all",constraint_all)


In [ ]:
print("lambda_all",lambda_all)


In [ ]:
print("q_critic_all",q_critic_all)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume these variables are returned by your agent.learn() call:
# rewards_all, average_runtime, kpis_list, observations_ep = agent.learn(episodes=5, ...)

# --- Process Cumulative Rewards ---
# Compute the cumulative reward for each episode (summing over timesteps)
#cumulative_rewards = [sum(episode_rewards) for episode_rewards in rewards_all]
cumulative_rewards = [
    sum(sum(agent_rewards) for agent_rewards in episode_rewards)
    for episode_rewards in rewards_all
]

# Plot Cumulative Reward per Episode
plt.figure(figsize=(10, 6))
plt.plot(range(len(cumulative_rewards)), cumulative_rewards, marker='o', linestyle='-', color='b')
plt.xlabel('Episode')
plt.ylabel('Cumulative Reward')
plt.title('Cumulative Reward per Episode')
plt.grid(True)
plt.show()




In [ ]:
kpis_list

In [ ]:
rewards_all

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Let's assume rewards_all is structured as:
# rewards_all[episode][time_step][agent]
# For example, if there are 3 agents and 5 episodes, each episode might look like:
# [[r1_t1, r2_t1, r3_t1], [r1_t2, r2_t2, r3_t2], ... ]

# First, determine the number of agents by checking one time step from one episode.
# (Assuming there's at least one episode and one time step.)
num_agents = len(rewards_all[0][0])

# For each episode, we want to compute the cumulative reward for each agent.
# This creates a list (or array) of shape (num_episodes, num_agents)
cumulative_rewards_per_agent = []
for episode_rewards in rewards_all:
    # episode_rewards is a list over time steps; convert it to a numpy array for easy summing:
    # Shape will be (num_time_steps, num_agents)
    episode_array = np.array(episode_rewards)
    # Sum over the time steps (axis=0) to get one reward per agent.
    cumulative_rewards = np.sum(episode_array, axis=0)
    cumulative_rewards_per_agent.append(cumulative_rewards)

# Convert to a numpy array of shape (num_episodes, num_agents)
cumulative_rewards_per_agent = np.array(cumulative_rewards_per_agent)

# Create a DataFrame for easier plotting if desired
episodes = np.arange(1, cumulative_rewards_per_agent.shape[0] + 1)
df = pd.DataFrame(cumulative_rewards_per_agent, index=episodes,
                  columns=[f'Agent {i+1}' for i in range(num_agents)])
print(df)

# Now, plot the cumulative reward per episode for each agent.
plt.figure(figsize=(10, 6))
for agent in df.columns:
    plt.plot(df.index, df[agent], marker='o', label=agent)

plt.xlabel('Episode')
plt.ylabel('Cumulative Reward')
plt.title('Cumulative Reward per Episode (Per Agent)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
print("Episode rewards_ep shape:", len(episode_rewards))
if len(episode_rewards) > 0:
    print("Time step 0 reward:", episode_rewards[0])



In [ ]:
for ep_i, ep_rewards in enumerate(rewards_all):
    arr = np.array(ep_rewards)
    sums = np.sum(arr, axis=0)
    print(f"Episode {ep_i}, final sums per agent: {sums}")


In [ ]:
# Each episode: a list of time-step rewards
# Each time step: a list of agent rewards
episode_array = np.array(episode_rewards)  # shape (time_steps, num_agents)
cumulative_rewards = np.sum(episode_array, axis=0)  # shape (num_agents,)
cumulative_rewards

In [ ]:
# --- Display Average Runtime ---
print(f"Average Prediction Runtime per Time Step: {average_runtime:.4f} seconds")

# --- Process and Plot KPI Data (if available) ---
# For example, suppose each KPI DataFrame has a column for a KPI called 'TotalCost'
if kpis_list and isinstance(kpis_list[0], pd.DataFrame):
    # Let's extract the 'TotalCost' KPI across episodes, if present.
    kpi_name = 'carbon_emissions_total'
    kpi_values = []
    for kpi_df in kpis_list:
        if kpi_name in kpi_df.columns:
            # If multiple rows exist, you might want to take a mean or the first value
            kpi_values.append(kpi_df[kpi_name].iloc[0])
        else:
            kpi_values.append(None)
    
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(kpi_values)), kpi_values, marker='o', linestyle='-', color='r')
    plt.xlabel('Episode')
    plt.ylabel(kpi_name)
    plt.title(f'{kpi_name} per Episode')
    plt.grid(True)
    plt.show()

# --- Optionally, inspect observations or other metrics ---
# For example, print the number of observations recorded
print(f"Total observation records collected: {len(observations_ep)}")

In [ ]:
reward_function = V2GPenaltyReward
env = CityLearnEnv(schema_test, central_agent=False, reward_function=reward_function)
observations = env.reset()
for time_step in range(env.time_steps - 1):
    actions = model.predict(observations)
    next_observations, _, _, _ = env.step(actions)

    for building_idx, building in enumerate(env.buildings):
        # Access the chargers in the building
        for charger in building.chargers:
            connected_ev = charger.connected_ev
            violations = []
            action = actions[building_idx][1]  #EV-related action is the second action.

            if connected_ev:
                current_soc = connected_ev.battery.soc[time_step]
                min_battery_soc = connected_ev.min_battery_soc
                max_battery_capacity = connected_ev.battery.capacity

                # 1. Discharging SOC=0 Violation 
                if current_soc == 0 and action < 0:
                    violations.append("Discharging when SOC is 0")
                    violations_count["Discharging when SOC is 0"] += 1

                # 2. Overcharging Violation
                if current_soc >= max_battery_capacity and action > 0:
                    violations.append("Overcharging Violation")
                    violations_count["Overcharging Violation"] += 1

                # 4. Charging Power Violation
                real_power = action * charger.nominal_power  # Corrected: Scaling by nominal power
                if real_power > charger.max_charging_power or real_power < charger.min_charging_power:
                    violations.append("Charging Power Violation")
                    violations_count["Charging Power Violation"] += 1

            else:
                # 3. Charging with No EV Connected
                if action > 0:
                    violations.append("Charging with No EV Connected")
                    violations_count["Charging with No EV Connected"] += 1

                # 5. Discharge During Idle
                if action < 0:
                    violations.append("Discharge During Idle")
                    violations_count["Discharge During Idle"] += 1
            # if building.electrical_storage_soc == 0 and action < 0:
            #   violations.append("Discharging Electrical Storage is 0")
            #   violations_count["Discharging Electrical Storage is 0"] += 1

            # Log the data
            log_entry = {
                "time_step": time_step,
                "building_id": building_idx + 1,
                "charger_id": charger.charger_id,
                "connected_ev": connected_ev.name if connected_ev else None,
                "connected_ev_soc": current_soc if connected_ev else None,
                "action": action,
                "constraint_violations": violations
            }
            log_data.append(log_entry)

    observations = next_observations

# Save Logs to File
log_df = pd.DataFrame(log_data)
log_df.to_csv("logs/constraint_logs_NEWALGO.csv", index=False)



In [ ]:
# Report Violations
total_violations = sum(violations_count.values())
print(f"Total Violations: {total_violations}")
print("Individual Contribution to Total Violations:")
for violation, count in violations_count.items():
    percentage = (count / total_violations * 100) if total_violations > 0 else 0
    print(f"{violation}: {count} ({percentage:.2f}%)")

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from typing import Union
import time
import os
from pathlib import Path

def learn(
        self, episodes: int = None, keep_env_history: bool = None, env_history_directory: Union[str, Path] = None, 
        deterministic: bool = None, deterministic_finish: bool = None, logging_level: int = None
    ):
    """Train agent.

    Parameters
    ----------
    episodes: int, default: 1
        Number of training episodes.
    keep_env_history: bool, default: False
        Whether to store environment state at the end of each episode.
    env_history_directory: Union[str, Path], optional
        Directory to save environment history.
    deterministic: bool, default: False
        Whether to use deterministic actions.
    deterministic_finish: bool, default: False
        Whether to use deterministic actions in the final episode.
    logging_level: int, default: 30
        Logging level for debugging.

    Returns
    -------
    rewards_all, average_runtime, kpis_list, observations_ep
    """
    episodes = 1 if episodes is None else episodes
    keep_env_history = False if keep_env_history is None else keep_env_history
    deterministic_finish = False if deterministic_finish is None else deterministic_finish
    deterministic = False if deterministic is None else deterministic
    self.__set_logger(logging_level)

    # Setup environment history directory if needed
    if keep_env_history:
        env_history_directory = Path(f'citylearn_learning_{self.env.uid}') if env_history_directory is None else env_history_directory
        os.makedirs(env_history_directory, exist_ok=True)
    
    # Create a TensorBoard writer (you can adjust the log directory as needed)
    writer = SummaryWriter(log_dir=f'logs/{self.env.uid}')

    rewards_all = []
    individual_runtimes_predict = []
    average_runtime = 0
    kpis_list = []
    observations_ep = []

    for episode in range(episodes):
        # Determine whether to use deterministic actions for this episode
        deterministic_episode = deterministic or (deterministic_finish and episode >= episodes - 1)
        observations = self.env.reset()
        rewards_ep = []
        episode_constraint_costs = []  # To track constraint cost at each time step

        start_episode_time = time.time()  # for runtime logging per episode

        while not self.env.done:
            print("\n\n------TIME STEP------")
            print(f"Episode {episode} - Time step: {self.env.time_step}")

            observations_ep.append(observations)
            start_time = time.time()
            actions = self.predict(observations, deterministic=deterministic_episode)
            end_time = time.time()
            individual_runtimes_predict.append(end_time - start_time)

            # Log constraint cost for each agent at this time step
            for agent_idx, action in enumerate(actions):
                # Convert action to tensor and add a batch dimension
                action_tensor = torch.FloatTensor(action).unsqueeze(0).to(self.device)
                cost_tensor = self.compute_constraint_cost(agent_idx, [action_tensor])
                episode_constraint_costs.append(cost_tensor.item())

            next_observations, rewards, _, _ = self.env.step(actions)
            rewards_ep.append(rewards)

            # Optionally, you can log other time-step metrics here
            observations = next_observations

        # Calculate episode-level metrics:
        total_reward = np.sum(rewards_ep)
        avg_constraint_cost = np.mean(episode_constraint_costs) if episode_constraint_costs else 0.0
        episode_runtime = time.time() - start_episode_time

        rewards_all.append(total_reward)
        kpis = self.env.evaluate().pivot(index='cost_function', columns='name', values='value')
        kpis = kpis.dropna(how='all')
        kpis_list.append(kpis)

        # Log metrics to TensorBoard
        writer.add_scalar("Episode/TotalReward", total_reward, episode)
        writer.add_scalar("Episode/AvgConstraintCost", avg_constraint_cost, episode)
        writer.add_scalar("Episode/Lagrangian", self.lagrangian.item(), episode)
        writer.add_scalar("Episode/EpisodeRuntime", episode_runtime, episode)

        print(f"Episode {episode} finished: Total Reward = {total_reward}, "
              f"Avg Constraint Cost = {avg_constraint_cost}, "
              f"Lagrangian = {self.lagrangian.item()}")
        print("KPIs:\n", kpis)

        # Save episode's environment history if required
        if keep_env_history:
            self.__save_env(episode, env_history_directory)

    average_runtime = sum(individual_runtimes_predict) / len(individual_runtimes_predict) if individual_runtimes_predict else 0.0

    writer.close()  # Ensure to close the writer after training
    return rewards_all, average_runtime, kpis_list, observations_ep


In [ ]:
from torch.utils.tensorboard import SummaryWriter
import time
import os
from pathlib import Path
import numpy as np

def learn(
        self, episodes: int = None, keep_env_history: bool = None, env_history_directory: Union[str, Path] = None, 
        deterministic: bool = None, deterministic_finish: bool = None, logging_level: int = None
    ):
    """
    Train agent and track multiple evaluation metrics using TensorBoard.

    Metrics tracked:
        - Bellman loss for Q–critic and constraint critic.
        - Cumulative Reward per episode.
        - Learning Curve (total reward across episodes).
        - Violation Frequency: For each agent, frequency = (# violations) / (# chargers).
        - Violation Magnitude: Average deviation beyond limits per violation.
        - Lagrangian Parameter Behavior.

    Returns:
        rewards_all, average_runtime, kpis_list, observations_ep
    """
    episodes = 1 if episodes is None else episodes
    keep_env_history = False if keep_env_history is None else keep_env_history
    deterministic_finish = False if deterministic_finish is None else deterministic_finish
    deterministic = False if deterministic is None else deterministic
    self.__set_logger(logging_level)

    if keep_env_history:
        env_history_directory = Path(f'citylearn_learning_{self.env.uid}') if env_history_directory is None else env_history_directory
        os.makedirs(env_history_directory, exist_ok=True)

    # Create a TensorBoard SummaryWriter
    writer = SummaryWriter(log_dir=f'logs/{self.env.uid}')

    rewards_all = []
    individual_runtimes_predict = []
    kpis_list = []
    observations_ep = []

    for episode in range(episodes):
        # Decide whether to use deterministic actions for this episode
        deterministic_episode = deterministic or (deterministic_finish and episode >= episodes - 1)
        observations = self.env.reset()
        episode_rewards = []
        episode_q_losses = []           # Accumulate Q–critic losses over update calls
        episode_constraint_losses = []  # Accumulate constraint critic losses
        episode_violation_frequencies = []  # Per time step violation frequency
        episode_violation_magnitudes = []   # Per time step average violation magnitude

        start_episode_time = time.time()

        while not self.env.done:
            print("\n\n------TIME STEP------")
            print(f"Episode {episode} - Time step: {self.env.time_step}")

            observations_ep.append(observations)
            start_time = time.time()
            actions = self.predict(observations, deterministic=deterministic_episode)
            end_time = time.time()
            individual_runtimes_predict.append(end_time - start_time)

            # Compute violation metrics for each agent at this time step
            for agent_idx, action in enumerate(actions):
                # Convert action to a tensor (with batch dimension)
                action_tensor = torch.FloatTensor(action).unsqueeze(0).to(self.device)
                cost_tensor = self.compute_constraint_cost(agent_idx, [action_tensor])
                violation_count = cost_tensor.item()  # total violations for this agent's sample

                # Get building info for the agent to compute frequency and magnitude
                building = self.env.buildings[agent_idx]
                num_chargers = len(building.chargers) if building.chargers else 1
                violation_frequency = violation_count / num_chargers
                episode_violation_frequencies.append(violation_frequency)

                # Compute violation magnitude for this agent
                magnitude_sum = 0.0
                violation_instances = 0
                if building.chargers:
                    for j, charger in enumerate(building.chargers):
                        act_value = action[j]
                        real_power = act_value * charger.nominal_power
                        if real_power > charger.max_charging_power:
                            magnitude_sum += (real_power - charger.max_charging_power)
                            violation_instances += 1
                        elif real_power < charger.min_charging_power:
                            magnitude_sum += (charger.min_charging_power - real_power)
                            violation_instances += 1
                violation_magnitude = magnitude_sum / violation_instances if violation_instances > 0 else 0.0
                episode_violation_magnitudes.append(violation_magnitude)

            # Step the environment
            next_observations, rewards, _, _ = self.env.step(actions)
            episode_rewards.append(rewards)

            # Update model if not in deterministic evaluation
            if not deterministic_episode:
                # Update() is assumed to return a dict with 'q_loss' and 'constraint_loss'
                losses = self.update(observations, actions, rewards, next_observations, done=self.env.done)
                if losses is not None:
                    episode_q_losses.append(losses['q_loss'])
                    episode_constraint_losses.append(losses['constraint_loss'])
            observations = next_observations

        # End of episode: aggregate metrics
        total_reward = np.sum(episode_rewards)
        avg_q_loss = np.mean(episode_q_losses) if episode_q_losses else 0.0
        avg_constraint_loss = np.mean(episode_constraint_losses) if episode_constraint_losses else 0.0
        avg_violation_frequency = np.mean(episode_violation_frequencies) if episode_violation_frequencies else 0.0
        avg_violation_magnitude = np.mean(episode_violation_magnitudes) if episode_violation_magnitudes else 0.0
        lagrangian_value = self.lagrangian.item()
        episode_runtime = time.time() - start_episode_time

        rewards_all.append(total_reward)
        kpis = self.env.evaluate().pivot(index='cost_function', columns='name', values='value')
        kpis = kpis.dropna(how='all')
        kpis_list.append(kpis)

        # Log metrics to TensorBoard
        writer.add_scalar("Episode/TotalReward", total_reward, episode)
        writer.add_scalar("Episode/AvgQLoss", avg_q_loss, episode)
        writer.add_scalar("Episode/AvgConstraintLoss", avg_constraint_loss, episode)
        writer.add_scalar("Episode/AvgViolationFrequency", avg_violation_frequency, episode)
        writer.add_scalar("Episode/AvgViolationMagnitude", avg_violation_magnitude, episode)
        writer.add_scalar("Episode/Lagrangian", lagrangian_value, episode)
        writer.add_scalar("Episode/EpisodeRuntime", episode_runtime, episode)

        print(f"Episode {episode} finished:")
        print(f"  Total Reward           = {total_reward}")
        print(f"  Avg Q Loss             = {avg_q_loss}")
        print(f"  Avg Constraint Loss    = {avg_constraint_loss}")
        print(f"  Avg Violation Frequency= {avg_violation_frequency}")
        print(f"  Avg Violation Magnitude= {avg_violation_magnitude}")
        print(f"  Lagrangian             = {lagrangian_value}")
        print("KPIs:\n", kpis)

        if keep_env_history:
            self.__save_env(episode, env_history_directory)

    average_runtime = sum(individual_runtimes_predict) / len(individual_runtimes_predict) if individual_runtimes_predict else 0.0
    writer.close()
    return rewards_all, average_runtime, kpis_list, observations_ep


In [ ]:
import time
import os
from pathlib import Path
import numpy as np
import pandas as pd

def learn(
        self, episodes: int = None, keep_env_history: bool = None, env_history_directory: Union[str, Path] = None, 
        deterministic: bool = None, deterministic_finish: bool = None, logging_level: int = None
    ):
    """
    Train agent and collect evaluation metrics per episode.

    Metrics collected:
        - Total reward per episode.
        - Average Q Loss and Constraint Loss per episode.
        - Average violation frequency and violation magnitude per episode.
        - Lagrangian parameter value.
        - Episode runtime.

    Returns:
        metrics_df (pd.DataFrame): DataFrame with metrics per episode.
        rewards_all, average_runtime, kpis_list, observations_ep
    """
    episodes = 1 if episodes is None else episodes
    keep_env_history = False if keep_env_history is None else keep_env_history
    deterministic_finish = False if deterministic_finish is None else deterministic_finish
    deterministic = False if deterministic is None else deterministic
    self.__set_logger(logging_level)

    if keep_env_history:
        env_history_directory = Path(f'citylearn_learning_{self.env.uid}') if env_history_directory is None else env_history_directory
        os.makedirs(env_history_directory, exist_ok=True)

    rewards_all = []
    individual_runtimes_predict = []
    kpis_list = []
    observations_ep = []

    # Lists to store metrics per episode
    episodes_list = []
    total_rewards = []
    avg_q_losses = []
    avg_constraint_losses = []
    avg_violation_frequencies = []
    avg_violation_magnitudes = []
    lagrangian_values = []
    episode_runtimes = []

    for episode in range(episodes):
        episodes_list.append(episode)
        deterministic_episode = deterministic or (deterministic_finish and episode >= episodes - 1)
        observations = self.env.reset()
        episode_rewards = []
        episode_q_losses = []           # To store Q–critic losses for update calls
        episode_constraint_losses = []  # To store constraint critic losses
        episode_violation_frequencies = []  # Violation frequency per time step
        episode_violation_magnitudes = []   # Violation magnitude per time step

        start_episode_time = time.time()

        while not self.env.done:
            print("\n\n------TIME STEP------")
            print(f"Episode {episode} - Time step: {self.env.time_step}")

            observations_ep.append(observations)
            start_time = time.time()
            actions = self.predict(observations, deterministic=deterministic_episode)
            end_time = time.time()
            individual_runtimes_predict.append(end_time - start_time)

            # Compute violation metrics for each agent at this time step
            for agent_idx, action in enumerate(actions):
                # Convert action to tensor (with batch dimension)
                action_tensor = torch.FloatTensor(action).unsqueeze(0).to(self.device)
                cost_tensor = self.compute_constraint_cost(agent_idx, [action_tensor])
                violation_count = cost_tensor.item()  # total violations for this agent sample

                building = self.env.buildings[agent_idx]
                num_chargers = len(building.chargers) if building.chargers else 1
                violation_frequency = violation_count / num_chargers
                episode_violation_frequencies.append(violation_frequency)

                # Compute violation magnitude for this agent
                magnitude_sum = 0.0
                violation_instances = 0
                if building.chargers:
                    for j, charger in enumerate(building.chargers):
                        act_value = action[j]
                        real_power = act_value * charger.nominal_power
                        if real_power > charger.max_charging_power:
                            magnitude_sum += (real_power - charger.max_charging_power)
                            violation_instances += 1
                        elif real_power < charger.min_charging_power:
                            magnitude_sum += (charger.min_charging_power - real_power)
                            violation_instances += 1
                violation_magnitude = magnitude_sum / violation_instances if violation_instances > 0 else 0.0
                episode_violation_magnitudes.append(violation_magnitude)

            next_observations, rewards, _, _ = self.env.step(actions)
            episode_rewards.append(rewards)

            if not deterministic_episode:
                # update() is assumed to return a dict with keys 'q_loss' and 'constraint_loss'
                losses = self.update(observations, actions, rewards, next_observations, done=self.env.done)
                if losses is not None:
                    episode_q_losses.append(losses['q_loss'])
                    episode_constraint_losses.append(losses['constraint_loss'])
            observations = next_observations

        # End-of-episode metrics aggregation
        total_reward = np.sum(episode_rewards)
        avg_q_loss = np.mean(episode_q_losses) if episode_q_losses else 0.0
        avg_constraint_loss = np.mean(episode_constraint_losses) if episode_constraint_losses else 0.0
        avg_violation_frequency = np.mean(episode_violation_frequencies) if episode_violation_frequencies else 0.0
        avg_violation_magnitude = np.mean(episode_violation_magnitudes) if episode_violation_magnitudes else 0.0
        lagrangian_value = self.lagrangian.item()
        episode_runtime = time.time() - start_episode_time

        rewards_all.append(total_reward)
        kpis = self.env.evaluate().pivot(index='cost_function', columns='name', values='value')
        kpis = kpis.dropna(how='all')
        kpis_list.append(kpis)

        # Save metrics for this episode
        total_rewards.append(total_reward)
        avg_q_losses.append(avg_q_loss)
        avg_constraint_losses.append(avg_constraint_loss)
        avg_violation_frequencies.append(avg_violation_frequency)
        avg_violation_magnitudes.append(avg_violation_magnitude)
        lagrangian_values.append(lagrangian_value)
        episode_runtimes.append(episode_runtime)

        print(f"Episode {episode} finished:")
        print(f"  Total Reward           = {total_reward}")
        print(f"  Avg Q Loss             = {avg_q_loss}")
        print(f"  Avg Constraint Loss    = {avg_constraint_loss}")
        print(f"  Avg Violation Frequency= {avg_violation_frequency}")
        print(f"  Avg Violation Magnitude= {avg_violation_magnitude}")
        print(f"  Lagrangian             = {lagrangian_value}")

        if keep_env_history:
            self.__save_env(episode, env_history_directory)

    average_runtime = sum(individual_runtimes_predict) / len(individual_runtimes_predict) if individual_runtimes_predict else 0.0

    # Create a DataFrame from the collected metrics
    metrics_df = pd.DataFrame({
        'Episode': episodes_list,
        'TotalReward': total_rewards,
        'AvgQLoss': avg_q_losses,
        'AvgConstraintLoss': avg_constraint_losses,
        'AvgViolationFrequency': avg_violation_frequencies,
        'AvgViolationMagnitude': avg_violation_magnitudes,
        'Lagrangian': lagrangian_values,
        'EpisodeRuntime': episode_runtimes
    })

    return metrics_df, rewards_all, average_runtime, kpis_list, observations_ep


In [ ]:
import matplotlib.pyplot as plt

# Assume `metrics_df` is returned from the learn() method.
metrics_df, _, _, _, _ = model.learn(episodes=50, deterministic=False)

# Plot Total Reward per Episode
plt.figure(figsize=(10, 6))
plt.plot(metrics_df['Episode'], metrics_df['TotalReward'], marker='o', linestyle='-')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Total Reward per Episode')
plt.grid(True)
plt.show()

# Plot Bellman Losses for Q-Critic and Constraint Critic
plt.figure(figsize=(10, 6))
plt.plot(metrics_df['Episode'], metrics_df['AvgQLoss'], marker='o', linestyle='-', label='Avg Q Loss')
plt.plot(metrics_df['Episode'], metrics_df['AvgConstraintLoss'], marker='o', linestyle='-', label='Avg Constraint Loss')
plt.xlabel('Episode')
plt.ylabel('Loss')
plt.title('Bellman Losses per Episode')
plt.legend()
plt.grid(True)
plt.show()

# Plot Violation Frequency and Magnitude
plt.figure(figsize=(10, 6))
plt.plot(metrics_df['Episode'], metrics_df['AvgViolationFrequency'], marker='o', linestyle='-', label='Violation Frequency')
plt.plot(metrics_df['Episode'], metrics_df['AvgViolationMagnitude'], marker='o', linestyle='-', label='Violation Magnitude')
plt.xlabel('Episode')
plt.ylabel('Value')
plt.title('Constraint Violation Metrics per Episode')
plt.legend()
plt.grid(True)
plt.show()

# Plot Lagrangian Parameter Behavior
plt.figure(figsize=(10, 6))
plt.plot(metrics_df['Episode'], metrics_df['Lagrangian'], marker='o', linestyle='-')
plt.xlabel('Episode')
plt.ylabel('Lagrangian Value')
plt.title('Lagrangian Parameter per Episode')
plt.grid(True)
plt.show()


In [ ]:
!pip install gym 